<a href="https://colab.research.google.com/github/autinn/autoflow/blob/main/finetuning_deepseek_resume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 3.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56

In [ ]:
from unsloth import FastLanguageModel

# set model to censor-free deepseek-r1-1776
# max_seq_length: set max length of the input sequence to optimize memory usage and processing speed
# dtype: dont need to check the data type
# load_in_4bit: enhance inference, reduce memory usage, quantise model to 4bit precision

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# target_modules: specifies the list of names of spcific classes / models we want to apply LoRA adaptation

# wrap base model with LoRA fine-tuning layers
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # define size of low-rank matrices used in adaptation
    # specify which model layer LoRA should modify
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj', # Multi-Head Attention
                      'gate_proj', 'up_proj', 'down_proj'], # Feedforward Network
    lora_alpha = 16, # scaling factor for LoRA layers to prevent LoRA from overpowering the base model
    lora_dropout = 0, # prevent overfitting by randomly siabling connections during training
    # dropout = 0 => fully leverage LoRA without dropping neurons (as unsloth already did it)
    bias = 'none', # whether we need additional bias terms
    use_gradient_checkpointing = 'unsloth', # gradient checkpoint to save GPU
    random_state = 3927,
    use_rslora = False,
    loftq_config = None # no quantisation for fine-tuning because it reduces precision (need to retain high precision to learn new patterns)
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth applies LoRA to 32 Transformer layers
QKV layer (Query, Key, Value layers)
O layer (Output projection)
MLP layer (feedforward layers)

Data preparation

In [ ]:
# instructions - signify main query to the LLM (i.e. the question)
# response - signify the output, mention how reponse should be tailored to specific instruction

human_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{}

### Response:
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token

def formatting_human_prompts_func(examples):
  texts = []
  labels = []

  for resume, job_description, label in zip(examples['resume_text'], examples['job_description_text'], examples['label']):
    instruction = f"Does the following resume fit the job description? Answer with 'Fit' or 'No Fit'. \n\nResume:\n{resume}\n\nJob Description:\n{job_description}"

    text = f"""
    ### Instruction:\n\n{instruction}\n
    ### Response: \n{label}{EOS_TOKEN}
    """

    texts.append(text)
    labels.append(1 if label.lower() == "fit" else 0)

  return {"text": texts, "label": labels}

In [ ]:
from datasets import load_dataset
dataset = load_dataset("cnamuangtoun/resume-job-description-fit", split='train')
dataset = dataset.map(formatting_human_prompts_func, batched = True)

Generating train split:   0%|          | 0/6241 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1759 [00:00<?, ? examples/s]

Map:   0%|          | 0/6241 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

Dataset({
    features: ['resume_text', 'job_description_text', 'label', 'text'],
    num_rows: 6241
})


In [ ]:
for i in range(5):
  print(dataset[i]['text'])


    ### Instruction:

Does the following resume fit the job description? Answer with 'Fit' or 'No Fit'. 

Resume:
SummaryHighly motivated Sales Associate with extensive customer service and sales experience. Outgoing sales professional with track record of driving increased sales, improving buying experience and elevating company profile with target market.
Highlights-Soft Skills: Public Speaking, Public Relations, Team Building, Project Management, Procedure writing, Staff Supervision and Management, Ability to interface with professionals on all levels. Accomplishments, Honors, and Activities -Board of Directors Member for the Food Bank of Corpus Christi from November 2010 to April 2013. -Held Life Insurance License -Basketball Official (Referee) High School Varsity Level.
ExperienceAccountant,08/2014-05/2015Aspirus–Owen,WI,Perform daily and routine accounting functions for two main companies and five small royalty companies. Responsibilities include but are not limited to the follo

In [ ]:
print(dataset)

Dataset({
    features: ['resume_text', 'job_description_text', 'label', 'text'],
    num_rows: 6241
})


In [ ]:
for i in range(3):
  print(dataset[i])

{'resume_text': "SummaryHighly motivated Sales Associate with extensive customer service and sales experience. Outgoing sales professional with track record of driving increased sales, improving buying experience and elevating company profile with target market.\nHighlights-Soft Skills: Public Speaking, Public Relations, Team Building, Project Management, Procedure writing, Staff Supervision and Management, Ability to interface with professionals on all levels. Accomplishments, Honors, and Activities -Board of Directors Member for the Food Bank of Corpus Christi from November 2010 to April 2013. -Held Life Insurance License -Basketball Official (Referee) High School Varsity Level.\nExperienceAccountant,08/2014-05/2015Aspirus–Owen,WI,Perform daily and routine accounting functions for two main companies and five small royalty companies. Responsibilities include but are not limited to the following: Accounts Payable, Accounts Receivable, Manage and reconcile funds for multiple banks accou

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = 'text', # column name that contains the input text for training
    max_seq_length = 2048, # max length of each training example
    dataset_num_proc = 2, # use 2 CPU processes to load and process data faster
    packing = False, # make examples separately processed (whether to combine short sample into large samples)
    args = TrainingArguments(
        per_device_train_batch_size = 2, # training examples per batch (i.e. batch size) per GPU
        gradient_accumulation_steps = 4, # step size of grdient accumulation - gradients are accumulated over 4 steps before an update
        warmup_steps = 5, # first 5 steps use smaller learning rate to stablize training
        # num_train_epoches = 1,
        max_steps = 50, # number of total optimization steps
        learning_rate = 2e-4, # initial learning rate
        fp16 = not is_bfloat16_supported(), # use fp16 precision if GPU does not support bf16
        bf16 = is_bfloat16_supported(), # use bf16 if GPU supports
        logging_steps = 1, # log trainig metrics every step
        optim = 'adamw_8bit', # use adamw-8bit as optimizer to reduce memory usage
        weight_decay = 0.01, # apply L2 regularisation to prevent overfitting
        lr_scheduler_type = 'linear', # learning rate linearly decreases from initial value to 0
        seed = 3407,
        output_dir = 'outputs', # save fine-tuned model into outputs/ folder
        report_to = "none"
    )
)


Converting train dataset to ChatML (num_proc=2):   0%|          | 0/6241 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/6241 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/6241 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/6241 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,241 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 167,772,160


Step,Training Loss
1,3.400900
2,3.462800
3,3.450100
4,3.639800
5,3.383100
6,3.264500
7,3.052500
8,3.024800
9,3.197400
10,3.108700


In [ ]:
new_model_local = "deepseek-r1-resume"
model.save_pretrained(new_model_local)
tokenizer.save_pretrained(new_model_local)
model.save_pretrained_merged(new_model_local, tokenizer, save_method='merged_16bit')

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.64 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 28%|██▊       | 9/32 [00:00<00:01, 12.62it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [03:19<00:00,  6.24s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving deepseek-r1-resume/pytorch_model-00001-of-00004.bin...
Unsloth: Saving deepseek-r1-resume/pytorch_model-00002-of-00004.bin...
Unsloth: Saving deepseek-r1-resume/pytorch_model-00003-of-00004.bin...
Unsloth: Saving deepseek-r1-resume/pytorch_model-00004-of-00004.bin...
Done.


In [ ]:
resume_text = "Summary7+ years of experience as a BI developer with a proven track record in Business Intelligence (BI), Data Warehouse (DWH) and Data Analytics related consulting projects.Proven ability to identify business needs and develop valuable solutions to drive accuracy and process efficiency.Experienced in developing, implementing, documenting, monitoring and maintaining the data warehouse extracts, transformations and ETL process in various industries like Financial, Health Care and Retail Industry withSales, Marketing, Inventory Management, Supply Chain and Financedomains.Delivered BI reporting solutions in Power BI, Reporting Services (SSRS).Expertise inData Warehousing ArchitecturesincludingETLdesign,Staging,Transformations,Delta/change Datacapture,Star-Schemas, Cubes andHistoryloadingSkilled in writingT-SQL Queries, Dynamic-queries, sub-queries and joins for generatingStored Procedures, Triggers, User-defined Functions, Views and Cursors.Experience with all phases of software development life cycle (SDLC) and Agile methodologies.Have delivered as a team member, team lead and as an independent consultant on medium to large scale projects. SkillsData/Business/SystemsTools: MS SQL Server Integration(SSIS), Analysis Services(SSAS), MetlDatabases: SQL Server 2012/2016,Netezza 7.2, Oracle 11g, AzureDevelopment Skills: T-SQL, DAX, MDX, C#BI Reporting Tools: Power BI, Reporting Services Services (SSRS).Data Quality and StandardsAgile MethodologyProgramming Languages:T-SQL, C#, VB Scripting. EducationGMR Institute of TechnologyRajam, India,Expected in05/2013––Bachelor of Science:Information Technology-GPA: ExperienceHoneywell-Sr. Data Analyst/Support EngineerGladstone,MO,05/2020-CurrentClient:Citi Bank, Irving, TxWorking as Level2 Support Engineer for Data Pipelines the Retail Customer Applications and Azure Data Staging.Analyzed large amounts of data to identify trends and find patterns, signals and hidden stories within data.Developing and supporting DevOps Repos and code pipelines for all ETLs Tasks and JobsDevelopedStandardETLintegrationsto extract Community-Loan-Location data from Mainframes system and implemented Business Rules to confirm the data.Experience in creating and managing SSAS tabular models, creating dimension and fact tables.Responsible for data administration tasksmaintenance plans,performance tuning,backup, and securityfor MS SQL Server systems.Primarily involved in data migration using SQL, SQL Azure, Azure storage, and azure data factory, SSIS, Powershell.Integrated Custom Visuals based on business requirements using Power BI desktop.Developed complex SQL queries using stored procedures, common table expressions (CTEs), temporary table to support Power BI and SSRS reports.Developed complex calculated measures using Data Analysis Expression language (DAX).Embedded Power BI reports on the Salesforce portal page and also managed access of reports and data for individual users using Roles.Provided continued maintenance and development of bug fixes for the existing and new Power BI Reports.Butler Technical Group-Sr. BI Data DeveloperOpa Locka,FL,08/2018-05/2020Led the development of Data Mart for supply chain,marketing departments along with analysis of existing data warehouse for performance improvements.Worked on all phases of data warehouse development life cycle, from gathering requirements, implementation, testing, training and support.Interviewed Business Users of the Data Warehouse and the Business Analysts to understand and troubleshoot the existing bottlenecks and problem areas.Developed applications and designed processes for transformation and data management from company-wide databases.Developed automated data dictionary for Enterprise Data Warehouse using SSRS and SQL.Completed a thorough analysis of the issues with the cube processing and query processing to identify bottleneck for the existing facts and dimensions.Designed and developed ETL packages to facilitate incremental load process.Created aggregates, partitions, attribute relationships and user defined hierarchies with the goal to reduce the cube processing time and query processing time.Created stored procedures and performed index analysis for slow performing SSRS reports to bring down the report rendering time.Developed Enterprise wide cascading reports in SSRS that were used throughout the agency to monitor the performance of the cube, logging of the nightly ETL loads and also to monitor most active users of the cube.Created robust and high-performing ETL mappings usingSSISandMetl.Migrated existing self-service reports and adhoc reports to Power BI.Developed custom calculated measures using DAX in Power BI to satisfy business needs.Assisted users in accessing databases, troubleshooting malfunctions and removing system-based barriers to task completion.Kairos Technologies Inc-Sr. ETL DeveloperCity,STATE,11/2016-07/2018Client:Cvs Caremark, Irving, TxWorked on ambitious project to develop an Enterprise Wide Drug plan Record system data warehouse connecting Insurance companies and Drug Manufacturers for CVS Caremark.Worked with Business Intelligence Manager to plan BI strategies, create BI road map and budget plans to deliver on organizations Business Intelligence needs.Assessed the impact of current business processes on users and stakeholders.Developed multiple dashboards, analytical reports, KPIs and Interactive reports based on business requirements using SSRS to support executive business needs.Transferred data from various data sources/business systems including MS Excel, MS Access, Flat files,COSMOS etc. to SQL Server using SSIS.Implemented Query Optimization and performance tuning for slow performing SQL queries.Involved in writing complex T-SQL queries and stored procedures to perform data transformations and to support SSRS Reports.Involved in creating User Security and Roles in reporting services at both parent level and report level.Supported maintenance of Data Quality Service's (DQS) knowledge base and development of Master Data Service's database.Tata Consultancy Services-Business Data AnalystCity,STATE,07/2013-04/2016Client:Sun Trust Bank, USDesigned, Implemented, and aided End-to-End data migration project using Microsoft SSIS.Assisted in creation of internal applications for marketing and customer relations departments, including financial reporting tools and reports.Designed SSIS Packages to extract data from various data sources such as Excel spreadsheet, and flat files and load data into destination databases for further Data Analysis and Reporting.Performed rigorous business case analyses and proposed various process improvements.Used Excel and Access applications for visualizing the data functionality.Led an initiative to convert all excel files which were used in in-house time/expense reporting wherein I automated the process to find discrepancies in timesheets by creating tables, views using stored procedures.Client :Marks and Spencer,UKImproved business direction by prioritizing customers and implementing changes based on collected feedback.Analyzed open orders, backlog, and sales data to provide sales team with insights.Created tables and define their relations by using foreign key and primary key relationship.Involved in creating database objects using stored procedures, Triggers, Functions, Views and T-SQL Joins using T-SQL.Performed normalization of database to reduce redundancy and achieve maximum system performance.Wrote custom T-SQL stored procedures and triggers to improve performance, preserve referential integrity, and provide additional application functionalityWorked as ETL Developer for an End-to-End implementation of Operation Data Warehousing Project.Responsible for data integration and transformation using SSIS.Formulated and documented the physical ETL process design based on business requirements and system specifications with strong ETL design skills.Created source to target mappings, transformations, lookups, aggregations, expressionsInvolved in scheduling reports, creating snapshot reports and subscription for the reports Using SSRS."
job_description = "Key Responsibilities:Create intricate wiring networks and electrical subcomponents for both prototype systems and their seamless integration into fuel cell-powered equipment.Conduct comprehensive tests, diagnoses, and repairs at the level of individual components, subsystems, and the overall system.Collaborate on the generation of electrical schematics and detailed bills of materials.Participate in safety assessments to ensure alignment with safety protocols for prototypes and finalized systems.Interface with engineering teams to ensure compliance with product interface requirements.Ensure system longevity and reliability through effective maintenance planning and execution.Regularly calibrate and fine-tune electrical elements to uphold system precision and stability.Occasional travel to customer and partner locations.Fulfill additional responsibilities as assigned by management. Requirements:Demonstrate strong mechanical and electrical acumen along with exceptional teamwork capabilities.Minimum of 5 years of practical experience in electrical technician or related roles.Certified to work on high-voltage systems in accordance with NFPA 70E (or equivalent) training.Proficient verbal communication skills, vital for interactions with manufacturing, engineering, and other teams both directly and through digital channels.Proficiency in fabricating electrical harnesses, cables, and devices based on wire schematics.Skilled at debugging and troubleshooting mechanical and electrical systems.An Associate's or Bachelor's degree in Electrical Engineering is preferred.Exhibit passion for innovation, sustainability, and pushing the boundaries of agricultural technologies.Possess adaptability to excel within a dynamic, fast-paced work environment."

In [ ]:
def human_prompt(resume_text, job_description, label):
  instruction = f"Does the following resume fit the job description? Answer with 'Fit' or 'No Fit'. \n\nResume:\n{resume_text}\n\nJob Description:\n{job_description}"
  text = f"""
      ### Instruction:{instruction}
      ### Response: {label}
      """
  return text

In [ ]:
human_prompt(resume_text, job_description,"")

In [ ]:
FastLanguageModel.for_inference(model)

# convert query into structure prompt then tokenize the prompt
# return_tensors='pt' to make tokenizer return a pytorch tensor then load tensor to GPU using .to('cuda') to increase speed

inputs = tokenizer(human_prompt(resume_text, job_description,""), return_tensors = 'pt').to('cuda')

# generate reponse for query with max_tokens and use_cache to speed up generation
outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache=True)

# decode the response
tokenizer.batch_decode(outputs)

Model Evaluation

In [ ]:
test_dataset = load_dataset("cnamuangtoun/resume-job-description-fit", split='test')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, classification_report

def evaluate_model(model, tokenizer, test_dataset):
  predictions = []
  ground_truths = []

  for sample in test_dataset:
    resume_text = sample['resume_text']
    job_description = sample['job_description_text']
    ground_truth = sample['label']

    input_prompt = human_prompt(resume_text, job_description,"")
    inputs = tokenizer(input_prompt, return_tensors = 'pt').to('cuda')
    outputs = model.generate(**inputs, max_new_tokens = 10, use_cache=True)

    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).replace(input_prompt, "").strip()
    predictions.append(prediction)
    ground_truths.append(ground_truth)

  binary_predictions = [1 if pred == 'Fit' else 0 for pred in predictions]
  binary_ground_truths = [1 if gt == 'Fit' else 0 for gt in ground_truths]

  accuracy = accuracy_score(binary_ground_truths, binary_predictions)
  precision = precision_score(binary_ground_truths, binary_predictions)

  report = classification_report(binary_ground_truths, binary_predictions, target_names=["No Fit", "Fit"])

  return accuracy, precision, report


In [ ]:
evaluate_model(model, tokenizer, test_dataset)